# Controllable Agents for RAG

<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/agent/agent_runner/agent_runner_rag_controllable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Adding agentic capabilities on top of your RAG pipeline can allow you to reason over much more complex questions.

But a big pain point for agents is the **lack of steerability/transparency**. An agent may tackle a user query through chain-of-thought/planning, which requires repeated calls to an LLM. During this process it can be hard to inspect what's going on, or stop/correct execution in the middle.

This notebook shows you how to use our brand-new lower-level agent API, which allows controllable step-wise execution, on top of a RAG pipeline.

We showcase this over Wikipedia documents.

In [ ]:
!pip install llama-index

## Setup Data

Here we load a simple dataset of different cities from Wikipedia.

In [ ]:
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
    ServiceContext,
)
from llama_index.llms import OpenAI
from llama_index.tools import QueryEngineTool, ToolMetadata

In [ ]:
# llm = OpenAI(model="gpt-3.5-turbo")
llm = OpenAI(model="gpt-4-1106-preview")
service_context = ServiceContext.from_defaults(llm=llm)

### Download Data

In [ ]:
!mkdir -p 'data/10q/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10q/uber_10q_march_2022.pdf' -O 'data/10q/uber_10q_march_2022.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10q/uber_10q_june_2022.pdf' -O 'data/10q/uber_10q_june_2022.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10q/uber_10q_sept_2022.pdf' -O 'data/10q/uber_10q_sept_2022.pdf'

### Load data

In [ ]:
march_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_march_2022.pdf"]
).load_data()
june_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_june_2022.pdf"]
).load_data()
sept_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_sept_2022.pdf"]
).load_data()

### Build indices/query engines/tools

In [ ]:
import os


def get_tool(name, full_name, documents=None):
    if not os.path.exists(f"./data/{name}"):
        # build vector index
        vector_index = VectorStoreIndex.from_documents(
            documents, service_context=service_context
        )
        vector_index.storage_context.persist(persist_dir=f"./data/{name}")
    else:
        vector_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=f"./data/{name}"),
            service_context=service_context,
        )
    query_engine = vector_index.as_query_engine(similarity_top_k=3)
    query_engine_tool = QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name=name,
            description=(
                "Provides information about Uber quarterly financials ending"
                f" {full_name}"
            ),
        ),
    )
    return query_engine_tool

In [ ]:
march_tool = get_tool("march_2022", "March 2022", documents=march_2022)
june_tool = get_tool("june_2022", "June 2022", documents=june_2022)
sept_tool = get_tool("sept_2022", "September 2022", documents=sept_2022)

In [ ]:
query_engine_tools = [march_tool, june_tool, sept_tool]

## Setup Agent

In this section we define our tools and setup the agent.

In [ ]:
from llama_index.agent import (
    AgentRunner,
    OpenAIAgentWorker,
    OpenAIAgent,
    ReActAgent,
)
from llama_index.agent.openai.step import OpenAIAgentWorker

# openai_step_engine = OpenAIAgentWorker.from_tools(
#     query_engine_tools, llm=llm, verbose=True
# )
# agent = AgentRunner(openai_step_engine)
# # alternative
# agent = OpenAIAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

agent_llm = OpenAI(model="gpt-3.5-turbo")
# agent_llm = OpenAI(model="gpt-4-1106-preview")

agent = ReActAgent.from_tools(
    query_engine_tools, llm=agent_llm, verbose=True, max_iterations=20
)

## Run Some Queries

We now demonstrate the capabilities of our step-wise agent framework. 

We show how it can handle complex queries, both e2e as well as step by step. 

We can then show how we can steer the outputs.

### Out of the box

Calling `chat` will attempt to run the task end-to-end, and we notice that it only ends up calling one tool.

In [ ]:
response = agent.chat(
    "Analyze the changes in R&D expenditures and revenue in the past 3 quarters"
)

Added user message to memory: Analyze the changes in R&D expenditures and revenue in the past 3 quarters
Thought: To analyze the changes in R&D expenditures and revenue in the past 3 quarters, I can use the tools march_2022, june_2022, and sept_2022 to gather the financial information for each quarter.
Action: march_2022
Action Input: {'input': 'R&D expenditures, revenue'}
Observation: Research and development expenses for the three months ended March 31, 2022, were $587 million, which was a 14% increase from $515 million in the same period in 2021. Revenue for the three months ended March 31, 2022, was $6,854 million, a 136% increase from $2,903 million in the same period in 2021.
Thought: (Implicit) I can answer without any more tools!
Response: Observation: According to the information from the march_2022 tool, the research and development expenses for the three months ended March 31, 2022, were $587 million, which represents a 14% increase from $515 million in the same period in 20

In [ ]:
print(str(response))

### Test Step-Wise Execution

The end-to-end chat didn't work. Let's try to break it down step-by-step, and inject our own feedback if things are going wrong.

In [ ]:
# start task
task = agent.create_task(
    "Analyze the changes in R&D expenditures and revenue in the past 3 quarters"
)

This returns a `Task` object, which contains the `input`, additional state in `extra_state`, and other fields.

Now let's try executing a single step of this task.

In [ ]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Analyze the changes in R&D expenditures and revenue in the past 3 quarters
Thought: To analyze the changes in R&D expenditures and revenue in the past 3 quarters, I can use the tools that provide information about Uber's quarterly financials for each quarter.
Action: sept_22
Action Input: {'input': 'R&D expenditures, revenue'}
Observation: Research and development expenses for the three months ended September 30, 2022, were $760 million, which was a 54% increase from $493 million in the same period in 2021. For the nine months ended September 30, 2022, research and development expenses were $2,051 million, a 37% increase from $1,496 million in the same period in 2021. The percentage of revenue allocated to research and development was 9% for both the three and nine months ended September 30, 2022, compared to 10% and 13%, respectively, for the same periods in 2021.


In [ ]:
step_output = agent.run_step(task.task_id)

Thought: (Implicit) I can answer without any more tools!
Response: Observation: The research and development expenses for the three months ended September 30, 2022, were $760 million, which is a 54% increase from $493 million in the same period in 2021. For the nine months ended September 30, 2022, the research and development expenses were $2,051 million, a 37% increase from $1,496 million in the same period in 2021. The percentage of revenue allocated to research and development was 9% for both the three and nine months ended September 30, 2022, compared to 10% and 13%, respectively, for the same periods in 2021.


We run into the **same issue**. The query finished even though we haven't analyzed the docs yet! Can we add a user input? 

In [ ]:
step_output = agent.run_step(task.task_id, input="What about June?")

Added user message to memory: What about June?
Thought: To provide information about June, I can use the "june_22" tool.
Action: june_22
Action Input: {'input': 'R&D expenditures, revenue'}
Observation: Research and development expenses for the three months ended June 30, 2022, were $704 million, which was a 44% increase from $488 million in the same period in 2021. For the six months ended June 30, 2022, research and development expenses were $1,291 million, a 29% increase from $1,003 million in the same period in 2021. The percentage of revenue allocated to research and development decreased from 12% to 9% for the three-month period and from 15% to 9% for the six-month period year-over-year.


In [ ]:
print(step_output.is_last)

False


In [ ]:
step_output = agent.run_step(task.task_id, input="What about March?")

Added user message to memory: What about March?
Thought: To provide information about March, I can use the "march_22" tool.
Action: march_22
Action Input: {'input': 'R&D expenditures, revenue'}
Observation: Research and development expenses for the three months ended March 31, 2022, were $587 million, which was a 14% increase from $515 million in the same period in 2021. Revenue for the three months ended March 31, 2022, was $6,854 million, a 136% increase from $2,903 million in the same period in 2021.


In [ ]:
step_output = agent.run_step(task.task_id)

Thought: I have analyzed the changes in R&D expenditures and revenue for the past 3 quarters using the available tools.
Response: Here are the observations for the changes in R&D expenditures and revenue for the past 3 quarters:

- September 2022:
  - R&D expenses: $760 million (54% increase from September 2021)
  - Revenue allocation to R&D: 9% (same as September 2021)

- June 2022:
  - R&D expenses: $704 million (44% increase from June 2021)
  - Revenue allocation to R&D: 9% (decreased from 12% in June 2021)

- March 2022:
  - R&D expenses: $587 million (14% increase from March 2021)
  - Revenue: $6,854 million (136% increase from March 2021)

Please note that the revenue information is only available for the March 2022 quarter.

Let me know if there's anything else I can help with!


Since the steps look good, we are now ready to call `finalize_response`, get back our response.

This will also commit the task execution to the `memory` object present in our `agent_runner`. We can inspect it.

In [ ]:
response = agent.finalize_response(task.task_id)

In [ ]:
print(str(response))

Here are the observations for the changes in R&D expenditures and revenue for the past 3 quarters:

- September 2022:
  - R&D expenses: $760 million (54% increase from September 2021)
  - Revenue allocation to R&D: 9% (same as September 2021)

- June 2022:
  - R&D expenses: $704 million (44% increase from June 2021)
  - Revenue allocation to R&D: 9% (decreased from 12% in June 2021)

- March 2022:
  - R&D expenses: $587 million (14% increase from March 2021)
  - Revenue: $6,854 million (136% increase from March 2021)

Please note that the revenue information is only available for the March 2022 quarter.

Let me know if there's anything else I can help with!


## Setup Human In the Loop Chat

With these capabilities, it's easy to setup human-in-the-loop (or LLM-in-the-loop) feedback when interacting with an agent, especially for long-running tasks.

We setup a double-loop: one for the task (the user "chatting" with an agent), and the other to control the intermediate executions.

In [ ]:
agent_llm = OpenAI(model="gpt-3.5-turbo")
# agent_llm = OpenAI(model="gpt-4-1106-preview")

agent = ReActAgent.from_tools(
    query_engine_tools, llm=agent_llm, verbose=True, max_iterations=20
)

In [ ]:
def chat_repl(exit_when_done: bool = True):
    """Chat REPL.

    Args:
        exit_when_done(bool): if True, automatically exit when step is finished.
            Set to False if you want to keep going even if step is marked as finished by the agent.
            If False, you need to explicitly call "exit" to finalize a task execution.

    """
    task_message = None
    while task_message != "exit":
        task_message = input(">> Human: ")
        if task_message == "exit":
            break

        task = agent.create_task(task_message)

        response = None
        step_output = None
        message = None
        while message != "exit":
            if message is None or message == "":
                step_output = agent.run_step(task.task_id)
            else:
                step_output = agent.run_step(task.task_id, input=message)
            if exit_when_done and step_output.is_last:
                print(
                    ">> Task marked as finished by the agent, executing task execution."
                )
                break

            message = input(
                ">> Add feedback during step? (press enter/leave blank to continue, and type 'exit' to stop): "
            )
            if message == "exit":
                break

        if step_output is None:
            print(">> You haven't run the agent. Task is discarded.")
        elif not step_output.is_last:
            print(">> The agent hasn't finished yet. Task is discarded.")
        else:
            response = agent.finalize_response(task.task_id)
        print(f"Agent: {str(response)}")

In [ ]:
chat_repl()

>> Human:  What are the risk factors in the last two quarters?


Added user message to memory: What are the risk factors in the last two quarters?
Thought: I need to use a tool to help me answer the question.
Action: march_2022
Action Input: {'input': 'risk factors'}
Observation: The risk factors affecting the business include:

- Significant reliance on Gross Bookings from trips in large metropolitan areas, which may be negatively affected by various conditions including economic, social, weather, regulatory conditions, and circumstances like COVID-19.
- Potential failure to offer autonomous vehicle technologies on the platform, or offering technologies that may be inferior or perceived as less safe compared to competitors.
- Dependence on retaining and attracting high-quality personnel, with the risk that attrition or unsuccessful succession planning could adversely affect the business.
- Risks of security or data privacy breaches, unauthorized access, alteration, or destruction of proprietary, confidential, employee, or platform user data.
- Thre

>> Add feedback during step? (press enter/leave blank to continue, and type 'exit' to stop):  I meant June and September


Added user message to memory: I meant June and September
Thought: I need to use a tool to help me answer the question.
Action: june_2022
Action Input: {'input': 'risk factors'}
Observation: The risk factors include:

1. Impact of COVID-19 pandemic or future disease outbreaks on business partners and third-party vendors, potentially leading to adverse impacts on the company's business, financial performance, and stock price.
2. Economic conditions affecting discretionary consumer spending, which may lead to shifts in consumer behavior towards lower-cost alternatives or reduced usage of the company's services.
3. Increases in fuel, food, labor, energy, and other costs due to inflation and other factors, which could increase operating costs for drivers, merchants, and carriers, potentially reducing their activity on the platform.
4. Dependence on the performance and reliability of Internet, mobile, and other infrastructures that are not under the company's control.
5. Risks associated wit

>> Add feedback during step? (press enter/leave blank to continue, and type 'exit' to stop):  


Thought: I need to use a tool to help me answer the question.
Action: sept_2022
Action Input: {'input': 'risk factors'}
Observation: Risk factors include:

- Software releases causing interruptions or negative experiences for platform users, potentially leading to loss of users, revenue, and legal or regulatory issues.
- Risks associated with the use of artificial intelligence, such as flawed algorithms, biased datasets, and potential legal and reputational harm due to unintentional bias and discrimination.
- Climate change risks, including physical risks like extreme weather events and transitional risks such as market shifts toward electric vehicles and lower carbon business models.
- Regulatory requirements related to climate change and the company's own climate commitments, which require significant investment and may be impacted by external factors.
- General economic risks, such as the impact of contagious disease outbreaks like the COVID-19 pandemic, which can reduce demand for 

>> Add feedback during step? (press enter/leave blank to continue, and type 'exit' to stop):  


Thought: I can answer without using any more tools.
Response: The risk factors mentioned in both June and September include the impact of the COVID-19 pandemic or future disease outbreaks, economic conditions affecting consumer spending, increases in costs, dependence on external infrastructures, potential engagement in inappropriate activities by platform users, substantial investments in new technologies, concentration of Gross Bookings in large metropolitan areas, risks associated with autonomous vehicle technologies, the need to retain high-quality personnel, security and data privacy breaches, cyberattacks, climate change risks, dependence on third parties, the need for additional capital, risks related to acquisitions and integration, legal and regulatory risks, government regulation and oversight, risks related to data collection and processing, challenges in protecting intellectual property, volatility in stock price, risks in certain jurisdictions, and the impact of catastroph

>> Human:  exit
